# Modelling realistic climate change
In previous weeks, we have computed the Equilibrium Climate Sensitivity, which tells us how much warming will eventually result from a doubling of CO$_2$ (once the Earth system reaches equilibrium).  But we know that CO2 doesn't instantly double, and the climate takes some time to reach its equilibrium temperature.  In this notebook, we will simulate changes over time with Climlab, and then we will use another simulator called En-ROADS to explore global actions to mitigate CO$_2$.


Part 1 of this notebook is based on chapter 15 of [The Climate Laboratory](https://brian-rose.github.io/ClimateLaboratoryBook) by [Brian E. J. Rose](http://www.atmos.albany.edu/facstaff/brose/index.html), University at Albany.  
Part 2 uses tools from [Climate Interactive](https://www.climateinteractive.org/).

## Part 1: Exploring the rate of climate change

We know that the rate of climate change matters for how we experience that change.  So: how will the temperature change with a gradual increase in CO2 over time?

Here we are going to look at a simple climate model and think about relationships between climate sensitivity, ocean heat uptake, and warming over time.

## Two versions of Radiative-Convective Equilibrium with different climate sensitivities

We are going set up two different single-column model with different lapse rate feedbacks.

We begin by repeating the same setup we have done several times before, building a single-column RCM with prescribed water vapor profile.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import climlab

In [ ]:
## Do a hacky import of vertical levels and water vapor from CESM simulations
## See https://brian-rose.github.io/ClimateLaboratoryBook/courseware/radeq.html#water-vapor-data
lev_cesm = np.array([  3.544638,   7.388814,  13.967214,  23.944625,  37.23029 ,  53.114605,
        70.05915 ,  85.439115, 100.514695, 118.250335, 139.115395, 163.66207 ,
       192.539935, 226.513265, 266.481155, 313.501265, 368.81798 , 433.895225,
       510.455255, 600.5242  , 696.79629 , 787.70206 , 867.16076 , 929.648875,
       970.55483 , 992.5561  ])

qglobal_spec_humid = np.array([2.16104904e-06, 2.15879387e-06, 2.15121262e-06, 2.13630949e-06,
       2.12163684e-06, 2.11168002e-06, 2.09396914e-06, 2.10589390e-06,
       2.42166155e-06, 3.12595653e-06, 5.01369691e-06, 9.60746488e-06,
       2.08907654e-05, 4.78823747e-05, 1.05492451e-04, 2.11889055e-04,
       3.94176751e-04, 7.10734458e-04, 1.34192099e-03, 2.05153261e-03,
       3.16844784e-03, 4.96883408e-03, 6.62218037e-03, 8.38350326e-03,
       9.38620899e-03, 9.65030544e-03])

In [ ]:
#  Make a model on same vertical domain as the GCM
state = climlab.column_state(lev=lev_cesm, water_depth=2.5)
steps_per_year = 90
deltat = climlab.constants.seconds_per_year/steps_per_year
rad = climlab.radiation.RRTMG(name='Radiation',
                              state=state, 
                              specific_humidity=qglobal_spec_humid,
                              timestep = deltat,
                              albedo = 0.25,  # tuned to give reasonable ASR for reference cloud-free model
                             )
conv = climlab.convection.ConvectiveAdjustment(name='Convection',
                                               state=state,
                                               adj_lapse_rate=6.5,
                                               timestep=rad.timestep,)
rcm_control = climlab.couple([rad,conv], name='Radiative-Convective Model')

Integrate the control model out to equilibrium.

In [ ]:
rcm_control.integrate_years(5)
rcm_control.ASR - rcm_control.OLR

Now let's make two copies of this model and keep them in a list:

In [ ]:
slab_control = []
slab_control.append(rcm_control)
slab_control.append(climlab.process_like(rcm_control))

We are going to **double CO2 in both models** and label them as high and low sensitivity. We will build in **different feedbacks** into our two columns.

In [ ]:
slab_2x = []
for n in range(len(slab_control)):
    rcm_2xCO2 = climlab.process_like(rcm_control)
    rcm_2xCO2.subprocess['Radiation'].absorber_vmr['CO2'] *= 2.
    if n == 0:
        rcm_2xCO2.name = 'High-sensitivity RCM'
    elif n == 1:
        rcm_2xCO2.name = 'Low-sensitivity RCM'
    slab_2x.append(rcm_2xCO2)

We will implement a water vapor feedback as we have done before: by recomputing the specific humidity at every timestep using the current temperatures so that the **relative humidity stays fixed**.

We begin by computing the relative humidity profile from the control climate.

In [ ]:
#  actual specific humidity
q = rcm_control.subprocess['Radiation'].specific_humidity
#  saturation specific humidity (a function of temperature and pressure)
qsat = climlab.utils.thermo.qsat(rcm_control.Tatm, rcm_control.lev)
#  Relative humidity
rh = q/qsat

Now here is where our two models will differ:

We are going to assign them **different lapse rate feedbacks**.

**Question:** What is the lapse rate feedback?  Refresh your memory in Dessler Chapter 6 or in [this online textbook chapter](http://www.climate.be/textbook/chapter4_node7.html) by Goosse.

In [ ]:
## your answer here

We are going to assume 

$$ \Gamma = \Gamma_{ref} + \gamma * \Delta T_s $$

where $\Gamma_{ref} = 6.5 K/km$ is the atmospheric lapse rate in our control climate, and $\gamma$ is a number in units of km$^{-1}$ that determines how much the atmospheric lapse rate should change per degree warming.

We are going to investigate two different assumptions:

- temperatures decrease **more** with height under global warming, $\gamma = +0.3$ km$^{-1}$
- temperature decrease **less** with height under global warming, $\gamma = -0.3$ km$^{-1}$

In [ ]:
lapse_change_factor = [+0.3, -0.3]

**Question:** Based on the Goosse resource, which of these corresponds to a dominant influence of the tropics?  Which corresponds to dominant influence of the poles?

...*your response here*

We will set up our two models with different lapse-rate feedbacks and check the ECS for each one:

In [ ]:
for n in range(len(slab_2x)):
    rcm_2xCO2 = slab_2x[n]
    print('Integrating ' + rcm_2xCO2.name)
    for m in range(5 * steps_per_year):
        # At every timestep
        # we calculate the new saturation specific humidity for the new temperature
        #  and change the water vapor in the radiation model
        #  so that relative humidity is always the same
        qsat = climlab.utils.thermo.qsat(rcm_2xCO2.Tatm, rcm_2xCO2.lev)
        rcm_2xCO2.subprocess['Radiation'].specific_humidity[:] = rh * qsat
        #  We also adjust the critical lapse rate in our convection model
        DeltaTs = rcm_2xCO2.Ts - rcm_control.Ts
        rcm_2xCO2.subprocess['Convection'].adj_lapse_rate = 6.5 + lapse_change_factor[n]*DeltaTs
        rcm_2xCO2.step_forward()
    print('The TOA imbalance is %0.5f W/m2' %(rcm_2xCO2.ASR-rcm_2xCO2.OLR))
    print('The ECS is %0.3f K' %(rcm_2xCO2.Ts - rcm_control.Ts))
    print('')

So Model 0 (in which the lapse rates have gotten larger) is **more sensitive** than Model 1 (smaller lapse rates). It has a larger system gain, or a more positive overall climate feedback. 

## Time to reach equilibrium

These models reached their new equilibria in just a few years. Why is that? Because they have very little heat capacity:

In [ ]:
slab_control[0].depth_bounds

The "ocean" in these models is just a "slab" of water 2.5 meter deep.

That's all we need to calculate the equilibrium temperatures, but it tells us nothing about the timescales for climate change in the real world.

For this, we need a deep ocean that can **exchange heat with the surface**.

## Transient warming scenarios in column models with ocean heat uptake

We are now going to build two new models. The atmosphere (radiative-convective model) will be identical to the two "slab" models we just used. But these will be coupled to a **column of ocean water** 2000 m deep!

We will **parameterize the ocean heat uptake** as a diffusive mixing process. This means we are assuming that ocean dynamics result in a vertical mixing of heat from warm to cold temperatures, as you saw firsthand in our thermohaline tank experiment.

The following code will set this up for us.

**Exercise: identify where in the code below we are adding an ocean** -- you might compare this code with the radiative-convective models we've set up before.

In [ ]:
#  Create the domains
ocean_bounds = np.arange(0., 2010., 100.)
depthax = climlab.Axis(axis_type='depth', bounds=ocean_bounds)
ocean = climlab.domain.domain.Ocean(axes=depthax)
atm = slab_control[0].Tatm.domain

ocean_diff = [3.5E-4, 3.5E-4] ## thermal diffusivity for model 0 and model 1

#  List of deep ocean models
deep = []
for n in range(len(slab_control)):
    rcm_control = slab_control[n]
    #  Create the state variables
    Tinitial_ocean = rcm_control.Ts * np.ones(ocean.shape)
    Tocean = climlab.Field(Tinitial_ocean.copy(), domain=ocean)
    Tatm = climlab.Field(rcm_control.Tatm.copy(), domain=atm)

    #  Surface temperature Ts is the upper-most grid box of the ocean
    Ts = Tocean[0:1]
    atm_state = {'Tatm': Tatm, 'Ts': Ts}
    
    rad = climlab.radiation.RRTMG(name='Radiation',
                                  state=atm_state, 
                                  specific_humidity=qglobal_spec_humid,
                                  timestep = deltat,
                                  albedo = 0.25,  
                                 )
    conv = climlab.convection.ConvectiveAdjustment(name='Convection',
                                                   state=atm_state,
                                                   adj_lapse_rate=6.5,
                                                   timestep=rad.timestep,)

    model = rad + conv
    if n == 0:
        model.name = 'RCM with high sensitivity'
    elif n == 1:
        model.name = 'RCM with low sensitivity'
    model.set_state('Tocean', Tocean)
    diff = climlab.dynamics.Diffusion(state={'Tocean': model.Tocean}, 
                                K=ocean_diff[n], 
                                diffusion_axis='depth', 
                                timestep=deltat * 10,)
    model.add_subprocess('Ocean Heat Uptake', diff)
    print('')
    print(model)
    print('')
    deep.append(model)

## An idealized transient global warming scenario: CO2 increases by 1%/year to doubling.

Now consider the CO2 increase. In the real world, CO2 has been increasing every year since the beginning of industrialization. Future CO2 concentrations depend on collective choices made by human societies about how much fossil fuel to extract and burn.

We will set up a simple scenario. Suppose that CO2 increases by 1% of its existing concentration every year **until it reaches 2x its initial concentration**. This takes about 70 years.

After 70 years, we assume that all anthropogenic emissions, and CO2 concentration is **stabilized** at the 2x level.

What happens to the surface temperature?

How do the histories of surface and deep ocean temperature compare in our two models?

We are going to simulation **400 years of transient global warming** in the two models.

<div class="alert alert-success">
This code will take a long time to run! While it's running, we'll think about what the result might look like
</div>

In [ ]:
num_years = 400
years = np.arange(num_years+1)

Tsarray = []
Tocean = []
netrad = []
for n in range(len(deep)):
    thisTs = np.nan * np.zeros(num_years+1)
    thisnetrad = np.nan * np.zeros(num_years+1)
    thisTocean = np.nan * np.zeros((deep[n].Tocean.size, num_years+1))
    thisTs[0] = deep[n].Ts
    thisnetrad[0] = deep[n].ASR - deep[n].OLR
    thisTocean[:, 0] = deep[n].Tocean
    Tsarray.append(thisTs)
    Tocean.append(thisTocean)
    netrad.append(thisnetrad)
    
CO2initial = deep[0].subprocess['Radiation'].absorber_vmr['CO2']
CO2array = np.nan * np.zeros(num_years+1)
CO2array[0] = CO2initial * 1E6

#  Increase CO2 by 1% / year for 70 years (until doubled), and then hold constant
for y in range(num_years):
    if deep[0].subprocess['Radiation'].absorber_vmr['CO2'] < 2 * CO2initial:
        for model in deep:
            model.subprocess['Radiation'].absorber_vmr['CO2'] *= 1.01
    CO2array[y+1] = deep[0].subprocess['Radiation'].absorber_vmr['CO2'] * 1E6
    print('Year ', y+1, ', CO2 mixing ratio is ', CO2array[y+1],' ppm.')

    for n, model in enumerate(deep):
        for m in range(steps_per_year):            
            qsat = climlab.utils.thermo.qsat(model.Tatm, model.lev)
            model.subprocess['Radiation'].specific_humidity[:] = rh * qsat
            DeltaTs = model.Ts - slab_control[n].Ts
            model.subprocess['Convection'].adj_lapse_rate = 6.5 + lapse_change_factor[n]*DeltaTs
            model.step_forward()
            
            Tsarray[n][y+1] = model.Ts
            Tocean[n][:, y+1] = model.Tocean
            netrad[n][y+1] = model.ASR - model.OLR

In [ ]:
colorlist = ['b', 'r']
co2color = 'k'

num_axes = len(deep) + 1
fig, ax = plt.subplots(num_axes, figsize=(12,14))


ax[0].plot(CO2array, color=co2color)
ax[0].set_ylabel('CO2 (ppm)', color=co2color)
for tl in ax[0].get_yticklabels():
    tl.set_color(co2color)
ax[0].set_ylim(300., 1000.)

slab_names=('High ECS equilib temp', 'Low ECS equilib temp')
deeper_names =('High ECS, deep ocean transient', 'Low ECS, deep ocean transient')
for n, model in enumerate(slab_2x):
    ax[1].plot(model.Ts*np.ones_like(Tsarray[n]), '--', color=colorlist[n], label=slab_names[n])
for n, model in enumerate(deep):
    ax[1].plot(Tsarray[n], color=colorlist[n], label=deeper_names[n])
ax[1].set_ylabel('Surface temperature (K)')
ax[1].set_xlabel('Years')
ax[1].set_title('Transient warming scenario: 1%/year CO2 increase to doubling, followed by CO2 stabilization', fontsize=14)
ax[1].legend(loc='lower right')

for n, model in enumerate(deep):
    ax[2].plot(netrad[n], ':', color=colorlist[n])
ax[2].set_ylabel('TOA imbalance (W/m2)')
ax[2].set_ylim(0, 3)

The plots above show the surface and atmosphere.  How about the change in ocean temperature over time?

In [ ]:
colorlist = ['b', 'r']
co2color = 'k'

num_axes = len(deep)
fig, ax = plt.subplots(num_axes, figsize=(12,14))

deeper_names =('High ECS, deep ocean transient', 'Low ECS, deep ocean transient')
contour_levels = np.arange(-0.25, 3.25, 0.25)
for n in range(len(deep)):
    cax = ax[n].contourf(years, deep[n].depth, Tocean[n] - Tsarray[n][0], levels=contour_levels)
    ax[n].invert_yaxis()
    ax[n].set_ylabel('Depth (m)')
    ax[n].set_xlabel('Years')
    ax[n].set_title(deeper_names[n])
    
fig.subplots_adjust(bottom=0.12)
cbar_ax = fig.add_axes([0.25, 0.02, 0.5, 0.03])
fig.colorbar(cax, cax=cbar_ax, orientation='horizontal', label=r'Change in temperature [$^{\circ}$C]');

## Transient vs. equilibrium warming: interpretation

**Questions:** 
1. Describe how the CO2 concentration changes over the course of the simulation.  Is CO2 ever removed from the atmosphere?
2. After year 70, what happens to the top-of-atmosphere radiative forcing?  Why?
3. How much warming happens *after* CO$_2$ forcing stops, in each case?
4. Describe the pattern of temperature change in the ocean.  How long does it take for the temperature of the deep ocean to increase by 1$^{\circ}$C?  Compare this with the atmosphere.

*your responses here*

**Exercise:** copy over the definition of the deep-ocean model and adjust the thermal diffusivity to be higher, say 5.E-4.  This represents a more efficient uptake of heat by the ocean.  Re-run the simulation, create new plots, and analyse your results.  **How does ocean heat uptake affect the path of warming?**

**Reflect:** How might uncertainty in these two parameters -- the ECS and the ocean heat uptake -- affect our planning for future climate change?

*your response here*

# Part 2: Scenario simulation with En-ROADS

In all cases above, at the time of CO2 doubling the model has achieved only a fraction of its equilibrium surface warming.  The difference between the warming at year 70 (when emissions stop) and the equilibrium warming is called the **committed warming**. It is the global warming associated with CO2 emissions that are **already in the atmosphere**.

So, limiting future climate change depends on averting emissions today.  What are some of the global actions that could put a reasonable limit on future climate change?  You will use the En-ROADS simulator to explore.

### Your task: create an En-ROADS scenario that can mitigate global warming to the internationally agreed target of below 2°C by 2100.

#### Steps
1.	Head over to the [En-ROADS welcome page](https://www.climateinteractive.org/ourwork/guided-assignment-welcome/) and watch the introductory video to En-ROADS.
2.	Develop a scenario. Open up the En-ROADS simulator and use it to develop your vision of limiting global warming to less than 2°C (3.6°F). 
3.	Save your scenario link: Click on "Share Your Scenario" in the upper right-hand corner of En-ROADS, then choose "Copy Scenario Link."   
4.	Answer the questions. After developing your preferred scenario, write a concise response to the questions below. 

#### Questions

1.	**Your Plan:** Paste your scenario link and a screenshot of your final En-ROADS dashboard. What are the top 3-5 most important policies in your strategy? (For example, the most important sliders that you moved). 

...your response here

2.	**Political Feasibility:** To implement your proposals, what actions and priorities are needed over the next two years in businesses, civil society, government, and the public?  

...your response here

3.	**Winners/ Losers:** Who would be the biggest winners and losers globally in your proposed future? Think about nations, but also about specific populations, demographics, or social groups.

...your response here

4.	**Surprises:** What surprised you about the behavior of the energy and climate system as captured in En-ROADS? For example, what actions had a bigger or smaller effect than you thought?

...your response here

5.	**Hope:** What trends in the world give you hope that your proposals are possible?

...your response here

6.	**Personal Action:** What can you personally do to help create the necessary changes?

...your response here

____________

## Credits

The climlab portion of this notebook is based on one from [The Climate Laboratory](https://brian-rose.github.io/ClimateLaboratoryBook), an open-source textbook developed and maintained by [Brian E. J. Rose](http://www.atmos.albany.edu/facstaff/brose/index.html), University at Albany.